# Preparation

In [1]:
import os, time
import pandas as pd
import torch
device = torch.device('cuda')

!pip install -q bitsandbytes
!git clone https://github.com/jniimi/ensemble_inference
%cd ensemble_inference
import ensemble_inference as ens

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.0 MB/s eta 0:00:00
Cloning into 'ensemble_inference'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.

# Load Pretrained Model
You need to register Hugging Face login token to `HF_TOKEN` in Colab Secret or manually login with huggingface_cli to retrieve the gated pretrained model like Llama from HF.

In [2]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model, tokenizer = ens.load_model(model_id=model_id, load_in_4bit=True)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [3]:
input = {
    'example_text': "This restaurant is awesome!",
    'example_label': 5,
    'review_text': "The French restaurant I visited today was not very good because it took long to serve my dishes. But the food was not so bad. So, I will visit again."
}
prompt = ens.create_prompt(**input)
print(prompt)

### Instruction
You are a helpful assistant evaluating the review texts about the restaurant. Please evaluate the review text and assign an integer score ranging from 1 for the most negative comment to 5 for the most positive comment.

### Review 1
User review: This restaurant is awesome!
Output: 5

### Review 2
User review: The French restaurant I visited today was not very good because it took long to serve my dishes. But the food was not so bad. So, I will visit again.
Output: 


# Single inference

In [4]:
model_seed = 1
ens.single_inference(
    prompt=prompt,
    tokenizer=tokenizer, model=model,
    model_seed=model_seed, model_temperature=1.0,
    device=device
    )

/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


{'time0': 1.7552218437194824, 'pred0': 2}

# One sample with Ensemble Inference

In [5]:
model_seeds = [1, 2, 3, 4, 5]
res = ens.ensemble_inference(
    prompt=prompt,
    tokenizer=tokenizer, model=model,
    model_seeds=model_seeds,
    model_temperature=1.0,
    device=device
    )
res

,time0,pred0,time1,pred1,time2,pred2,time3,pred3,time4,pred4,total_time,pred
0,0.832162,2,0.803144,3,0.804645,2,0.799371,3,0.789331,2,4.028652,2
